<a href="https://colab.research.google.com/github/hsuanlion/RecommendationSystem/blob/main/week2/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 14:32:36--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.5’

All_Beauty.csv.5    100%[===================>]  14.78M  19.0MB/s    in 0.8s    

2022-01-02 14:32:37 (19.0 MB/s) - ‘All_Beauty.csv.5’ saved [15499476/15499476]

--2022-01-02 14:32:37--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.5’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.1MB/s    in 0.7s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [4]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [5]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 資料EDA

探索潛在作為tfidf運算矩陣的資料欄位分佈

In [7]:
metadata.main_cat.value_counts()

All Beauty    32892
Name: main_cat, dtype: int64

In [8]:
metadata.brand.value_counts()

                       15673
VAGA                     252
L'Oreal Paris            113
Philips Norelco           88
Gillette                  83
                       ...  
Tool Bench Hardware        1
Chicago Comb               1
Multi Oro                  1
Dr. Shrammek               1
Now Foods                  1
Name: brand, Length: 7863, dtype: int64

In [53]:
metadata['rank'].str.split(pat='in',expand=True)[1].str.replace(" \(","").value_counts()

 Beauty & Personal Care        31795
 Beauty &amp; Personal Care      585
 Grocery & Gourmet Food           36
 Health & Household               19
 Sports & Outdoors                 3
 Toys & Games                      3
 Health &amp; Household            2
 Baby                              2
 Grocery &amp; Gourmet Food        2
 Cloth                             2
 Home & Kitchen                    1
 Tools & Home Improvement          1
 Automotive                        1
Name: 1, dtype: int64

In [16]:
# 看description長度最長的index
df.description.apply(len).idxmax()

7078

In [17]:
len(df.description[7078])

44

In [18]:
df.description[7078]

['Crest 3D White High Performance Whitening for Men gives you noticeably whiter teeth in 10 days. In fact, youll start seeing results after only 3 days. Advanced Seal technology provides a form fit, no slip grip that molds and adheres to your teeth for a better whitening experience. Crest 3D White High Performance Whitening is the same great product as the Crest 3D White Advanced Vivid kit but with 10 strip pouches vs. 14. Reveal your whiter smile. Apply 3 easy steps to a Beautiful, Whiter smile.',
 '<div class="aplus">',
 'If you&#8217;re not whitening, you&#8217;re yellowing. From the coffee you brew to the wine you sip, stains build up daily to dull your white teeth and affect your overall appearance. So, while you&#8217;re splashing on cologne or rolling on deodorant, you should be teeth whitening with Crest&#174; 3D WhiteTM&nbsp;High Performance Whitening Whitestrips&#174;&nbsp;and change the way you think about your personal style.',
 '<b>High performance. Noticeable results.</b>

## 產生推薦

In [20]:
# 檢查testing data的users 沒有歷史資料個個數：佔93%以上
countU = 0
for user in users:
  if not ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist():
    countU +=1
print('no Historic UU:', countU, "total testing UU: ", len(users), "no Historic UU%: ", countU/len(users))

no Historic UU: 546 total testing UU:  584 no Historic UU%:  0.934931506849315


In [6]:
# 如果沒有歷史資料（即沒有input可以計算相似物件，也無法計算使用者相似度)，就用熱門商品
# 以下則將商品資料做一個整理
# 合併ratings & metadata_filtered
metadata['rank_n'] = pd.to_numeric(metadata['rank'].str.split('in',expand= True)[0].str.replace(',',''))

metadata_filterd = metadata[['asin', 'brand', 'title', 'price', 'rank', 'description','also_view','also_buy','rank_n']].copy()

metadata_filterd = metadata_filterd.set_index('asin')
metadata_filterd['review_counts'] = ratings.groupby('asin').size()
metadata_filterd['overall_mean'] = ratings.groupby('asin')['overall'].mean()
metadata_filterd['review_counts_train'] = ratings_trainings.groupby('asin').size()
metadata_filterd['overall_mean_train'] = ratings_trainings.groupby('asin')['overall'].mean()

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates(subset = ['title']).copy()
# df['rank_category'] = df['rank'].str.split(pat='in',expand=True)[1].str.replace(" \(","")
df['combine'] = df['title'] + " " + df['brand'].fillna("") # + " " + df["rank_category"].fillna("")
tf = TfidfVectorizer(analyzer='word',stop_words='english',min_df=0.001)
tfidf_matrix = tf.fit_transform(df['combine'])

# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['combine'])


# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品，當作input丟入recommend_item，產生推薦
# items 應是消費者購買過的商品清單
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [8]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    # version 1: recommend_items是使用物件的title當作計算相似度的基礎，input都是user有評比過的商品清單
    # 問題一：會出現部分user 推薦清單>10項，（如 ASGIVOW34XNQS, ACE1ZIOI3U6PY)
    # users: 是testing的user，如果在training時間段沒有買過東西，就會沒有推薦清單
    # recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    
    recommendations = {}

    for user in users:
      if not metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist():
        # 如果沒有歷史資料的新用戶，就推薦熱門評分前10大商品
        recommendations[user] = metadata_filterd.reset_index().sort_values('review_counts_train',ascending = False).drop_duplicates(subset = ['asin']).head(10)['asin'].to_list()
      else :
        recommendations[user] = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k)    

    # default
    # recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}

    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG',
  'B00VF344X0'],
 'A103T1QOGFCSEH': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG',
  'B00VF344X0'],
 'A106UKKSJ2KXPF': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG',
  'B00VF344X0'],
 'A10A7GV4D5A11V': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG',
  'B00VF344X0'],
 'A1119JJ37ZLB8R': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG',
  'B00VF344X0'],
 'A113UOOLBSZN52': ['B000FOI48G',
  'B000GLRREU',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006

## 結果評估

In [9]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.08135593220338982

In [ ]:
# v1: 如果指使用title當作特徵，recall = 0 (因為93%都是新用戶)
# v2: 7%使用content-based filtering (title + brand), 93%使用rule-based filtering, recall = 8.1% (0.08135593220338982)